In [ ]:

import xml.sax
import sys
sys.path.append('/content/drive/My Drive')
import collections
import os
import os.path
import re
import re
from collections import defaultdict
import Stemmer
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import timeit
from parsedata import Parser
import bz2
import glob


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# !pip3 install PyStemmer
# !pip3 install nltk

In [ ]:
curr_id = 1
id_titlemap = dict()
global_dict = dict() 
total_tokens = 0
# inverted_index_file_path = ""
index_file_cnt = 1



def createPageDict(title, infobox, body, category, link, references):
    pagedict = dict()
    
    # list format title, infobox, body, category, link, references
    # create local dictionary pagedict for entire page.. keep updating counts
    li = [0, 0, 0 ,0, 0 ,0]
    for i in title.keys():
        if i not in pagedict:
            pagedict[i]=[0, 0, 0 ,0, 0 ,0]
        pagedict[i][0]+=title[i]
            
    for i in infobox.keys():
        if i not in pagedict:
            pagedict[i]=[0, 0, 0 ,0, 0 ,0]
        pagedict[i][1]+=infobox[i]
    
    for i in body.keys():
        if i not in pagedict:
            pagedict[i]=[0, 0, 0 ,0, 0 ,0]
        pagedict[i][2]+=body[i]
            
    for i in category.keys():
        if i not in pagedict:
            pagedict[i]=[0, 0, 0 ,0, 0 ,0]
        pagedict[i][3]+=category[i]
            
    for i in link.keys():
        if i not in pagedict:
            pagedict[i]=[0, 0, 0 ,0, 0 ,0]
        pagedict[i][4]+=link[i]
            
    for i in references.keys():
        if i not in pagedict:
            pagedict[i]=[0, 0, 0 ,0, 0 ,0]
        pagedict[i][5]+=references[i]
        
    return pagedict

def mergeGlobalDict(currid, pagedict):
    global global_dict
    
    # if w already present in global dict then add new doc dictionary to it
    for w in pagedict:
        lt = pagedict[w]
        tmpdict = {}
        if w in global_dict:
            tmpdict = global_dict[w]
        tmpdict[currid] = lt
        global_dict[w] = tmpdict
      
    
def flushGlobalToFile():
    # format word:doc_id#tag_freq+tag_freq|doc_id#tag_freq+tag_freq
    # format word:d1#t2+i4|d3#t5+l4
    global global_dict
    global index_file_cnt
    inverted_index_file_path = '/content/drive/My Drive/IRE/IndexFiles/indexfile_'+str(index_file_cnt)+'.txt'
    index_file_cnt += 1
    taglist = ['t', 'i', 'b', 'c', 'l', 'r']
    with open(inverted_index_file_path,"a") as of:
        for word,docs in sorted(global_dict.items()):
            if not(re.match('^[a-zA-Z0-9]+$',word)) or re.match('^[0]+$',word):
                continue
            docdict = docs
            docfreq = []
            for k,v in sorted(docdict.items()):
                docdet = "d"+str(k)+"#"
                for i in range(6):
                    if v[i]>0:
                        docdet += taglist[i]+str(v[i])+"+"
                docdet = docdet[:-1]                             # remove last +
                docfreq.append(docdet)
            docfreqstr = '|'.join(docfreq)
            of.write(word+":"+docfreqstr+"\n")  
    of.close()         

def flushTitleToFile():
   global id_titlemap
   title_file = "/content/drive/My Drive/IRE/IndexFiles/id_title.txt"
   with open(title_file, "a") as of:
      for id in id_titlemap:
        of.write(str(id)+":"+id_titlemap[id]+'\n')
   of.close()

class XMLHandler(xml.sax.ContentHandler):
    
    global curr_id
    global id_titlemap
    
    def __init__(self):
        self.currenttag = ""
        self.title = ""
        self.text = ""
        self.obj = Parser()
        
    def startElement(self, tag, attributes):
        self.currenttag = tag
        if self.currenttag == "page":
            self.text = ""
            self.title = ""
            
    def characters(self, content):
        # collect data between <tag>..</tag>
        if self.currenttag == "title":
            self.title += content                   
        elif self.currenttag == "text":
            self.text += content

            
    def endElement(self, tag):
        global curr_id
        global global_dict
        global total_tokens
        if tag == "page":
            curr_id += 1
            if curr_id%50000 == 0:
              #write global dictionary to file
              flushGlobalToFile()
              flushTitleToFile()
              global_dict.clear()
              id_titlemap.clear()
              # sys.exit()
        if tag == "text":
            id_titlemap[curr_id] = self.title
            
            # parse and preprocess title --> returns dictionary for title
            processed_title, title_tokens = self.obj.preprocessTitle(self.title)
            total_tokens += title_tokens
            
            # parse and preprocess text --> returns dictionary for infobox, body, category, links, references
            processed_infobox, processed_body, processed_category, processed_links,  processed_references, text_tokens= self.obj.preprocessText(self.text)
            total_tokens += text_tokens
            
#             print("--------------------")
#             print(curr_id, "title ", processed_title)
#             print("infobox ", processed_infobox)
#             print("body ", processed_body)
#             print("category ", processed_category)
#             print("links ", processed_links)
#             print("references ", processed_references)
#             print("---------------------------")

            # combined page dictionary with title, infobox, body, category, links, references dict merged
            pagedict = createPageDict(processed_title, processed_infobox, processed_body, processed_category, processed_links, processed_references)
            
            #merge local dict with global dictionary
            mergeGlobalDict(curr_id, pagedict)
            # print('ended ' , curr_id)
            
        
        self.currenttag = ""
        





def createParser(xml_input):
    
    parserobj = xml.sax.make_parser()
    contenthandler = XMLHandler()
    
    # content handler for parser
    parserobj.setContentHandler(contenthandler)
    parserobj.parse(xml_input)
   




if (__name__=="__main__"):
    
    # global inverted_index_file_path
    filenames = glob.glob('/content/drive/My Drive/IRE/Phase2/*')
    for name in filenames:

      print(name)
      xml_input = bz2.BZ2File(name,'r')
      
      # print(xml_input)
      # inverted_index_file_path = 'index_file'+ str(cnt) + '.txt'
      # stat_file_path = 'stat.txt'
      start = timeit.default_timer()
      createParser(xml_input)
      
      #write remaning global dict to file
      global global_dict
      if bool(global_dict):
          flushGlobalToFile()
          flushTitleToFile()
          global_dict.clear()
          id_titlemap.clear()
      # with open(stat_file_path,"a") as of:
      #     of.write(str(total_tokens)+'\n')
      #     of.write(str(len(global_dict))+'\n')
      # of.close()
      
      stop = timeit.default_timer()
      print('Time: ', stop - start)
      # print()





/content/drive/My Drive/IRE/Phase2/enwiki-20200801-pages-articles-multistream23.xml-p29823661p30503450.bz2
Time:  693.2824634630001
/content/drive/My Drive/IRE/Phase2/enwiki-20200801-pages-articles-multistream1.xml-p1p30303.bz2
Time:  463.4843610300004
/content/drive/My Drive/IRE/Phase2/enwiki-20200801-pages-articles-multistream2.xml-p30304p88444.bz2
Time:  543.9788476939998
/content/drive/My Drive/IRE/Phase2/enwiki-20200801-pages-articles-multistream3.xml-p88445p200509.bz2
Time:  805.1485778019996
/content/drive/My Drive/IRE/Phase2/enwiki-20200801-pages-articles-multistream4.xml-p200510p352689.bz2
Time:  718.6307796990004
/content/drive/My Drive/IRE/Phase2/enwiki-20200801-pages-articles-multistream5.xml-p352690p565313.bz2
Time:  809.5716987490005
/content/drive/My Drive/IRE/Phase2/enwiki-20200801-pages-articles-multistream6.xml-p565314p892912.bz2
Time:  1014.2305999999999
/content/drive/My Drive/IRE/Phase2/enwiki-20200801-pages-articles-multistream7.xml-p892913p1268691.bz2
Time:  961.

KeyboardInterrupt: ignored

In [ ]:
print(currid)

NameError: ignored

In [1]:
!tail -3 '/content/drive/My Drive/IRE/IndexFiles/id_title.txt'

9484616:Category:Buildings and structures completed in 1907
9484617:Mort, où est ta victoire?
9484618:Jens Josef
